# 라이브러리 설치

In [ ]:
!pip install fitz tools frontend pdfplumber
!mkdir static

# 파일 다운로드

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import os

# 셀레니움 웹드라이버 설정
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # 브라우저 창을 열지 않고 실행
driver = webdriver.Chrome(options=options)

# 로그인 페이지 접근 및 로그인 처리
driver.get("https://www.milkt.co.kr/Login/Login")
time.sleep(1)  # 페이지 로드 대기

# 로그인 요소 찾기 및 로그인 정보 입력
driver.find_element(By.ID, 'input_id').send_keys("")  # 아이디
driver.find_element(By.ID, 'input_pw').send_keys("")  # 비번
driver.find_element(By.ID, 'btn_login').click()
time.sleep(2)  # 로그인 후 로딩 대기

# 팝업 창 닫기 처리
main_window = driver.current_window_handle
for handle in driver.window_handles:
    if handle != main_window:
        driver.switch_to.window(handle)
        time.sleep(2)  # 팝업 창에서 로딩이 필요한 경우 대기
        driver.close()  # 팝업 창 닫기
        driver.switch_to.window(main_window)  # 원래 창으로 돌아오기

try:
    close_button = driver.find_element(By.CLASS_NAME, "btn_close")  # 실제 팝업 닫기 버튼 클래스명 필요
    close_button.click()
    print("레이어 팝업이 닫혔습니다.")
except:
    print("레이어 팝업이 없습니다.")

# 페이지 순회를 통한 파일 다운로드
for page in range(1, 7):  # 페이지는 1에서 6까지
    url = f"https://www.milkt.co.kr/LectureMaterial/LectureMaterial?inPageNo={page}&intPageSize=30&strCode=A04&strSubCode=A0402&inGrade=0&inTerm=0&strlectTit=&strlectCont="
    driver.get(url)
    time.sleep(2)  # 페이지 로드 대기

    # 팝업 창 닫기 처리 (반복적 팝업이 있을 경우)
    main_window = driver.current_window_handle
    for handle in driver.window_handles:
        if handle != main_window:
            driver.switch_to.window(handle)
            time.sleep(1)  # 팝업 창에서 로딩이 필요한 경우 대기
            driver.close()  # 팝업 창 닫기
            driver.switch_to.window(main_window)  # 원래 창으로 돌아오기

    # 파일 다운로드 처리
    # 여기에 파일 다운로드 로직을 추가합니다. 예를 들어:
    # 파일 다운로드 버튼을 찾아 클릭하는 로직 작성

    # 페이지 이동 완료 후 알림
    print(f"{page} 페이지에서 파일 다운로드를 완료했습니다.")

# 드라이버 종료
driver.quit()


# 단어, 단어 뜻, 품사, 예문, 예문 뜻 추출
        

In [ ]:
#  최종 코드
import pdfplumber
import fitz  # PyMuPDF
import os
import re
import json

def extract_lesson_titles(page):
    text = page.extract_text()
    lesson_titles = re.findall(r'Lesson \d+\. (.+?)\s*\n', text)
    return lesson_titles

def extract_data_from_table(row):
    example_sentence, example_meaning = "", ""
    if re.search(r'[A-Za-z]+.*?[.!?]\s+[가-힣]', row[6]):
        parts = re.split(r'(?<=[.!?])\s+(?=[가-힣])', row[6], 1)
    elif re.search(r'[A-Za-z]+\s+[가-힣]', row[6]):
        parts = re.split(r'(?<=[A-Za-z])\s+(?=[가-힣])', row[6], 1)
    elif "\n" in row[6]:
        parts = row[6].split("\n", 1)
    else:
        parts = [row[6].strip(), ""]

    example_sentence = remove_newlines(parts[0].strip())
    mixed = extract_sentences_with_korean(example_sentence)
    example_meaning = remove_newlines(parts[1].strip()) if len(parts) > 1 else ""
    if example_meaning.startswith('씨'):
        concat = example_sentence + example_meaning
        example_meaning = extract_sentences_with_korean(concat)[0]
        example_sentence = concat.replace(example_meaning, '')
    if len(mixed) > 0: 
        if mixed[0] in example_sentence:
            example_sentence = example_sentence.replace(mixed[0], '')
            example_meaning = mixed[0] + ' ' + example_meaning

    pos_match = re.search(r'(\w+)\)\s*', row[3])
    if pos_match:
        part_of_speech = pos_match.group(1)
        meaning_text = re.sub(r'\s*\(\w+\)\s*', '', row[3]).strip()
    else:
        part_of_speech = ""
        meaning_text = row[3].strip()

    return {
        "단어": remove_newlines(row[0].strip()),
        "단어 뜻": meaning_text.replace(part_of_speech + ')', '').strip(),
        "품사": part_of_speech,
        "예문": example_sentence.strip(),
        "예문 뜻": example_meaning.strip(),
    }

def remove_newlines(text):
    return text.replace("\n", " ").replace("\r", "")

def extract_sentences_with_korean(text):
    sentences = re.split(r'(?<=[.?!])\s+', text)
    korean_sentences = [sentence for sentence in sentences if re.search(r'[가-힣]', sentence)]
    return korean_sentences

def safe_filename(filename):
    return re.sub(r'[\/:*?"<>|]', "", filename)

def pdf_to_json(pdf_path, output_directory):
    with pdfplumber.open(pdf_path) as pdf:
        for page_number, page in enumerate(pdf.pages):
            lesson_titles = extract_lesson_titles(page)
            title_text = " - ".join(lesson_titles)
            safe_title_text = safe_filename(title_text)
            data = []
            tables = page.extract_tables()
            for table in tables:
                for row in table:
                    if len(row) > 0:
                        entry = extract_data_from_table(row)
                        if any(entry.values()):
                            data.append(entry)
            output_file = os.path.join(output_directory, f"{os.path.basename(pdf_path).replace('.pdf', '')}_page_{page_number + 1}_{safe_title_text}.json")
            with open(output_file, "w", encoding="utf-8") as json_file:
                json.dump(data, json_file, ensure_ascii=False, indent=4)
            print(f"Parsed data for {pdf_path} page {page_number + 1}:", data)

def clean_text(text):
    return text.strip()

def format_sentence(sentence):
    if re.search(r'\s햄', sentence):
        sentence = sentence.replace(".  햄", "\n햄")
    return sentence

def extract_words_from_pdf_by_page(pdf_path, output_dir="output_pages"):
    document = fitz.open(pdf_path)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for page_num in range(document.page_count):
        page = document.load_page(page_num)
        text = format_sentence(page.get_text().replace('"', '')).replace(". 나", "\n나").replace("\n는", "는").replace(". 우", "\n우").replace("\n만", "만").replace("\n다", "다").replace("\n있다", "있다").replace("\n은편에", "은편에")
        
        # Lesson 제목을 뽑아서 파일 이름에 포함
        lesson_title = re.search(r'Lesson \d+\. (.+?)\s*\n', text)
        title_text = lesson_title.group(1) if lesson_title else f"Page_{page_num + 1}"
        safe_title_text = safe_filename(title_text)

        pattern = r"(?P<단어>\w+)\s+(?P<품사>[a-z.]+)\s+(?P<단어_뜻>.+?)\n(?P<예문>.+?)\n(?P<예문_뜻>.+?)\n"
        if page_num == 12 or page_num ==23:
            pattern = r"(?P<단어>\w+(?:\s\w+)*)\s+(?P<품사>[a-z.]+)\s+(?P<단어_뜻>.+?)\n(?P<예문>.+?)\n(?P<예문_뜻>.+?)\n"
        matches = re.finditer(pattern, text, re.MULTILINE)
        
        page_words = []
        for match in matches:
            word_info = {
                "단어": clean_text(match.group("단어")),
                "단어 뜻": clean_text(match.group("단어_뜻")),
                "품사": clean_text(match.group("품사")),
                "예문": clean_text(match.group("예문")),
                "예문 뜻": clean_text(match.group("예문_뜻"))
            }
            if word_info["단어"] and word_info["예문"]:
                page_words.append(word_info)
        
        # 파일 이름 예: fly1_page_1_라이트 형제.json
        output_file = os.path.join(output_dir, f"{os.path.basename(pdf_path).replace('.pdf', '')}_page_{page_num + 1}_{safe_title_text}.json")
        with open(output_file, "w", encoding="utf-8") as json_file:
            json.dump(page_words, json_file, ensure_ascii=False, indent=4)
    
    document.close()
    print(f"페이지별 JSON 파일이 '{output_dir}' 폴더에 저장되었습니다.")

def process_pdfs(pdf_directory: str, output_directory: str):
    # PDF 폴더 확인
    if not os.path.exists(pdf_directory):
        print("PDF 폴더가 없습니다.")
        return
        
    # 추출 폴더 생성 및 JSON 파일 생성
    os.makedirs(output_directory, exist_ok=True)
    for pdf_file in os.listdir(pdf_directory):
        pdf_path = os.path.join(pdf_directory, pdf_file)
        if pdf_file.endswith(".pdf"):
            if pdf_file == "beyond.pdf":
                print(f"{pdf_file}를 처리하는 중...")
                extract_words_from_pdf_by_page(pdf_path, output_dir=output_directory)
            else:
                pdf_to_json(pdf_path, output_directory)

# pdf 경로
PDF_DIR = "./pdf"

# 추출 경로
OUT_DIR = "./json_words"

# PDF 파일 처리
process_pdfs(PDF_DIR, OUT_DIR)
